In [6]:
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import fft
from scipy import signal as sig
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import math
import matplotlib.pyplot as plt
from cmath import phase
from sklearn.model_selection import train_test_split

In [7]:
class ForecastModel:
    def __init__(self, frequency, prominance):
        self.model = None
        self.model_simple = None
        self.fourier_model = None
        self.prominance = prominance
        self.frequency = frequency
        self.model_metrics = {} #R-Squared, Mse
        self.model_simple_metrics = {}

    def predict(self, x, simple=False):

        #Find Corresponding model. Trying switching to simple if no complex model exists
        if simple:
            if not self.model_simple:
                print(f'Simple Model for {self.name} does not exist. You will need to fit the model.')
                return
            else:
                model = self.model_simple
        else:
            if not self.model:
                print(f'Model for {self.name} does not exist. You will need to fit the model.\nTrying to use simple model.')
                if not self.model_simple:
                    print(f'Simple Model for {self.name} does not exist. You will need to fit the model.')
                    return
                else:
                    model = self.model_simple
            else:
                model = self.model
        
        # revise
        x = x if simple else self.get_fourier(x=x)
        self.validate_input(x, fourier=~simple)

        station_exits = model.predict(x)
        return station_exits
    

    #Take Fourier Model apply to every time in x adding a fourier column to x
    def get_fourier(self, x):
        array = np.zeros(len(x))
        for key in self.fourier_model.keys():
            a = self.fourier_model[key]['amplitude']
            w = self.fourier_model[key]['frequency']
            p = self.fourier_model[key]['phase']
            array +=  x['time_elapsed'].apply(
                lambda t: a * math.cos(w*t + p))
        x.loc[:,'ft_sum'] = array
        # x['ft_sum'] = pd.DataFrame([row.amplitude * np.cos(row.wavelength*x['time_elapsed'] + row.phase) 
        #                             for index, row in self.fourier_model.iterrows()]).sum(axis=0)
        return x 


    #Generating dictionary of fourier terms
    def generate_fourier(self, residuals):
        fft_output = fft.fft(residuals)
        power = np.abs(fft_output)
        freq = fft.fftfreq(len(residuals))

        mask = freq >= 0
        freq = freq[mask]
        power = power[mask]

        peaks = sig.find_peaks(power[freq >=0], prominence=self.prominance)[0]
        peak_freq =  freq[peaks]
        peak_power = power[peaks]

        output = pd.DataFrame()
        output['index'] = peaks
        output['frequency'] = peak_freq
        output['amplitude'] = peak_power
        output['period'] = 1 / peak_freq 
        output['fft'] = fft_output[peaks]
        output = output.sort_values('amplitude', ascending=False)

        # filtered_fft_output = np.array([f if i in list(output['index']) else 0 for i, f in enumerate(fft_output)])

        fourier_terms = pd.DataFrame()
        fourier_terms['fft'] = output['fft']
        fourier_terms['frequency'] = output['frequency']
        fourier_terms['amplitude'] = fourier_terms.fft.apply(lambda z: abs(z)) 
        fourier_terms['phase'] = fourier_terms.fft.apply(lambda z: phase(z))
        fourier_terms.sort_values(by=['amplitude'], ascending=[0])

        # Create some helpful labels (FT_1..FT_N)
        fourier_terms['label'] = list(map(lambda n : 'FT_{}'.format(n), range(1, len(fourier_terms) + 1)))
        # Turn our dataframe into a dictionary for easy lookup
        fourier_terms = fourier_terms.set_index('label')
        fourier_terms_dict = fourier_terms.to_dict('index')

        for key in fourier_terms_dict.keys():
            fourier_terms_dict[key]['amplitude'] = fourier_terms_dict[key]['amplitude']
            fourier_terms_dict[key]['frequency']  = 2 * math.pi * (fourier_terms_dict[key]['frequency'] / self.frequency)
            fourier_terms_dict[key]['phase'] = fourier_terms_dict[key]['phase']

        # fourier_terms = pd.DataFrame(fourier_terms_dict).transpose()
        # fourier_terms_df = pd.DataFrame()
        
        # fourier_terms_df['amplitude'] = fourier_terms.amplitude,
        # fourier_terms_df['wavelength'] =    2 * math.pi * (fourier_terms['frequency'] / 24)
        # fourier_terms_df['phase'] = fourier_terms['phase']
            # columns=['amplitude, wavelength, phase'])
        return fourier_terms_dict

        
    def validate_input(self, x, fourier=False):
        expected_columns = ['time_elapsed', 'hospitalizations'] if ~fourier else ['time_elapsed', 'hospitalizations', 'ft_sum']
        assert list(x.columns) == expected_columns, f'input does not match expected {expected_columns} received {x.columns}'
    

    # split data for fitting
    def split_data(self, x, y, split=False):
        self.validate_input(x)
        try:
            if split:
                x1, x2, y1, y2 = train_test_split(x, y, test_size=split, random_state=42)
                # split = int(len(x)*split)
                # x1 = x.iloc[:-split]
                # x2 = x.iloc[-split:]
                # y1 = y.iloc[:-split]
                # y2 = y.iloc[-split:]
            else:
                x1 = x
                x2 = x
                y1 = y
                y2 = y
        except:
            print(f'Error splitting Data for {self.name}. Split equals {split}')
            raise Exception
        return x1, x2, y1, y2

    def fit_model_simple(self, x, y, for_residuals=False, split=False, return_predictions=False):
        x1, x2, y1, y2 = self.split_data(x=x, y=y, split=split)
        
        model_simple = LinearRegression()
        model_simple.fit(x1, y1) 

        if for_residuals:
            residuals = np.array(y - model_simple.predict(x))
            return residuals

        self.model_simple = model_simple
        y_pred = self.predict(x=x2, simple=True)
        self.model_simple_metrics['mse'] = mean_squared_error(y2, y_pred)
        self.model_simple_metrics['r_squared'] = r2_score(y2, y_pred)

        if return_predictions:
            return self.predict(x, simple=True)
        return


    def fit_model(self, x, y, split=False, return_predictions=False):
        x1, x2, y1, y2 = self.split_data(x=x, y=y, split=split)     

        residuals = self.fit_model_simple(x,y,for_residuals=True, split=False) 
        self.fourier_model =  self.generate_fourier(residuals)
        x1, x2 = [self.get_fourier(x=x1), self.get_fourier(x=x2)]

        self.model = LinearRegression()
        self.model.fit(x1, y1) 
        y_pred = self.model.predict(x2)
        self.model_metrics['mse'] = mean_squared_error(y2, y_pred)
        self.model_metrics['r_squared'] = r2_score(y2, y_pred)
        if return_predictions:
            return self.predict(x, simple=False)


class Station(ForecastModel):
    def __init__(self, name, lines, borough, zip_code, type_model = 'Station', frequency= 24, prominance = 10**4):
        ForecastModel.__init__(self, frequency, prominance)
        self.name = name
        self.lines = [line for line in lines]
        self.zip_code = zip_code
        self.borough = borough
        self.type= type_model

class Line(ForecastModel):
    def __init__(self, name, stations, boroughs, zip_codes, type_model = 'Line', frequency= 24, prominance = 10**4):
        ForecastModel.__init__(self, frequency, prominance)
        self.name = name
        self.stations = stations
        self.zip_codes = zip_codes
        self.boroughs = boroughs
        self.type= type_model

class Zip_Code(ForecastModel):
    def __init__(self, name, stations, lines, boroughs, type_model = 'Zip Code', frequency= 24, prominance = 10**4):
        ForecastModel.__init__(self, frequency, prominance)
        self.name = name
        self.stations = stations
        self.boroughs = boroughs
        self.lines = lines
        self.type= type_model

class Borough(ForecastModel):
    def __init__(self, name, stations, lines, zip_codes, type_model = 'Borough', frequency= 24, prominance = 10**4):
        ForecastModel.__init__(self, frequency, prominance)
        self.name = name
        self.stations = stations
        self.zip_codes = zip_codes
        self.lines = lines
        self.type= type_model

class City(ForecastModel):
    def __init__(self, name = 'Full Model', type_model = 'Full Model', frequency= 24, prominance = 10**4):
            ForecastModel.__init__(self, frequency, prominance)
            self.name = name
            self.type= type_model

class MTA_Model:
    def __init__(self, name='Parent Model'):
        self.name = name
        self.stations = {}
        self.zip_codes = {}
        self.lines = {}
        self.boroughs = {}
        self.city = None
        pass

    def checkpoint(self, path='./MTA_Model_Checkpoint'):
        file = open(path, 'wb')
        pickle.dump(self, file)
        file.close()
        print(f'checkpoint: {path}')

    def get_metrics(self):
        metrics = {}
        if self.city: 
            metrics['city'] = self.city.model_metrics
            metrics['city']['type'] = self.city.type
        for dictionary in [self.stations, self.boroughs,self.zip_codes,self.lines]:
            if dictionary:
                for key in dictionary.keys():
                    metrics[key] = dictionary[key].model_metrics
                    metrics[key]['type'] = dictionary[key].type
        return metrics


    def generate_models(self, data, frequency= 24, prominance = 10**4, simple=False,
                        split=False, city=True, boroughs=True, stations=True, lines=True, zip_codes=True):
        #Generate Model for the city as whole
        if city:
            grouped_data = data.groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()
            X = grouped_data[['time_elapsed', 'hospitalizations']]
            Y = grouped_data['target']
            self.city = City(frequency= frequency, prominance = prominance)
            self.city.fit_model(x=X,y=Y,split=split)
            self.checkpoint(path='./MTA_Model_Checkpoint_City')

        #Generate Model for the boroughs
        if boroughs:
            boroughs_list = data['borough'].unique()
            grouped_data = data.groupby(['borough','time_elapsed']).agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()
            for bor in boroughs_list:
                mask = grouped_data.borough == bor
                full_mask = data.borough == bor
                station_list = data[full_mask].station.unique()
                zip_list = data[full_mask].zip_code.unique()
                line_list = set([char for char in ''.join(data[full_mask].lines.astype(str))])
                X = grouped_data[mask][['time_elapsed', 'hospitalizations']]
                Y = grouped_data[mask]['target']
                self.boroughs[bor] = Borough(name=bor, stations=station_list, zip_codes=zip_list, lines=line_list,frequency= frequency, prominance = prominance)
                self.boroughs[bor].fit_model(x=X,y=Y, split=split)
            self.checkpoint(path='./MTA_Model_Checkpoint_Boroughs')

        #Generate Model for the zip codes
        if zip_codes:
            zip_list = data['zip_code'].unique()
            grouped_data = data.groupby(['zip_code','time_elapsed']).agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()
            for zip in zip_list:
                mask = grouped_data.zip_code == zip
                full_mask = data.zip_code == zip
                station_list = data[full_mask].station.unique()
                borough_list = data[full_mask].borough.unique()
                line_list = set([char for char in ''.join(data[full_mask].lines.astype(str))])
                X = grouped_data[mask][['time_elapsed', 'hospitalizations']]
                Y = grouped_data[mask]['target']
                self.zip_codes[zip] = Zip_Code(name=zip, stations=station_list, boroughs=borough_list, lines=line_list,frequency= frequency, prominance = prominance)
                self.zip_codes[zip].fit_model(x=X,y=Y,split=split)
            self.checkpoint(path='./MTA_Model_Checkpoint_ZipCode')

        #Generate Model for the Lines
        if lines:
            lines_list = set([char for char in ''.join(data.lines.astype(str))])
            # grouped_data = data.groupby(['borough','time_elapsed']).sum().reset_index()
            for line in lines_list:
                full_mask = data.lines.str.contains(line)
                station_list = data[full_mask].station.unique()
                zip_list = data[full_mask].zip_code.unique()
                borough_list = data[full_mask].borough.unique()
                filtered_data = data[full_mask].groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()
                X = filtered_data[['time_elapsed', 'hospitalizations']]
                Y = filtered_data['target']
                self.lines[line] = Line(name=line, stations=station_list, zip_codes=zip_list, boroughs=borough_list,frequency= frequency, prominance = prominance)
                self.lines[line].fit_model(x=X,y=Y,split=split)
            self.checkpoint(path='./MTA_Model_Checkpoint_Lines')

        #Generate Model for the boroughs
        if stations:
            station_list = data['station'].unique()
            grouped_data = data.groupby(['station','time_elapsed']).agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()
            for station in station_list:
                mask = grouped_data.station == station
                full_mask = data.station == station
                borough_list = data[full_mask].borough.unique()
                zip_list = data[full_mask].zip_code.unique()
                line_list = set([char for char in ''.join(data[full_mask].lines.astype(str))])
                X = grouped_data[mask][['time_elapsed', 'hospitalizations']]
                Y = grouped_data[mask]['target']
                self.stations[station] = Station(name=station, borough=borough_list, zip_code=zip_list, lines=line_list,frequency= frequency, prominance = prominance)
                self.stations[station].fit_model(x=X,y=Y,split=split)
            self.checkpoint(path='./MTA_Model_Checkpoint_Stations')
        self.checkpoint(path='./MTA_Model_Checkpoint_Final')

    def resolve_missing_time(self, x):
        output = {}
        for key in MTA_MODEL.stations.keys():
            time_map = pd.DataFrame(df['time_elapsed'].unique(), columns=['time_elapsed'])
            out = MTA_MODEL.stations[key].predict(x=df[df['station']==key].groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()[['time_elapsed','hospitalizations']], simple=False)
            if len(out) < len(time_map):
                test = df[df['station']==key].groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()[['time_elapsed','hospitalizations']]
                X = pd.merge(time_map, test, on='time_elapsed', how='left')
                X.fillna(df.hospitalizations.mean(),inplace=True)
                out = MTA_MODEL.stations[key].predict(x=X, simple=False)
            output[key] = out

    def predict_group(self, x, missing_values = 0, simple=False, city=False, stations=False, lines= False, boroughs=False, zip_codes=False):
        output = pd.DataFrame()
        time_map = pd.DataFrame(df['time_elapsed'].unique(), columns=['time_elapsed'])
        prediction_columns = ['time_elapsed','hospitalizations']
        groupings = []
        groupings.append('time_elapsed')
        # groupings = ['station', 'lines', 'borough', 'zip_code', 'time_elapsed']
        if stations: groupings.append('station')
        if lines: groupings.append('lines')
        if zip_codes: groupings.append('zip_code')
        if boroughs: groupings.append('borough')
        

        filled_x = x.groupby(groupings).agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()
        filled_x = pd.merge(time_map, filled_x, on='time_elapsed', how='left')
        filled_x.fillna(missing_values,inplace=True)

        # predictions_to_run = [cat for cat in [city, stations, lines, boroughs, zip_codes] if cat]
        # #Validate the alignment of X for concating outputs
        # def validate_data(x, out, group, criteria):
        #     if len(out) < len(time_map):
        #         filled_x = x[x[group] == criteria].groupby(['station', 'time_elapsed']).resent_index()
        #         filled_x.fillna(filled_x.hospitalizations.mean(),inplace=True)
        #         return filled_x
        #     return False
        if city:
            output['city'] = self.city.predict(filled_x.groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()[prediction_columns], simple=simple)
        if stations:
            for key in self.stations.keys():
                output[key] =   self.stations[key].predict(x=filled_x[filled_x['station'] == key][prediction_columns]
                                .groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index(), simple=simple)
        if lines:
            for key in self.lines.keys():
                output[key] = self.lines[key].predict(x=filled_x[filled_x['lines'].str.contains(key)][prediction_columns]
                            .groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index(), simple=simple)
        if boroughs:
            for key in self.boroughs.keys():
                output[key] = self.boroughs[key].predict(x=filled_x[filled_x['borough'] == key][prediction_columns]
                            .groupby('time_elapsed').agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index(), simple=simple)
        if zip_codes:
            for key in self.zip_codes.keys():
                output[key] = self.zip_codes[key].predict(x=filled_x[filled_x['station'] == key][prediction_columns]
                            .groupby('time_elapsed').agg({'time_elapsed':'max', 'hospitalizations': 'max', 'target': 'sum'}).reset_index(), simple=simple)
        
        return output

In [8]:
def pickle_file(obj, path):
    file = open(path, 'wb')
    pickle.dump(obj, file)
    file.close()

def load_pickle(path):
    file = open(path, 'rb')
    obj = pickle.load(file)
    file.close()
    return obj
df = load_pickle('./data_merged/mta_station_broughs_cases')
# df = df.groupby('DATE','STATION', 'ZIP_CODE', 'LINENAME').sum().reset_index()
df['WEEK_YEAR']= pd.to_datetime(df["DATE"].dt.strftime('%Y-%m'))
# df = df[(df.DATE >= '2020-04-01') & (df.DATE <= '2022-01-01')]
df = df[df.DATE >= '2022-05-01']
# df['HOURS_ELAPSE'] = (df.DATE - pd.to_datetime('2020-04-01')).astype('timedelta64[h]')
df['HOURS_ELAPSE'] = (df.DATE - df.DATE.min()).astype('timedelta64[h]')


df['zip_code'] = df.ZIP_CODE.astype(str)
df['hospitalizations'] = df.HOSPITALIZED_COUNT
df['borough'] = df.BOROUGH
# df['time_elapsed'] = (df.DATE - pd.to_datetime('2020-04-01')).astype('timedelta64[h]')
df['time_elapsed'] = (df.DATE - df.DATE.min()).astype('timedelta64[h]')
df['target'] = df.DAILY_EXITS + df.DAILY_ENTRIES
df['station'] = df.STATION
df['lines'] = df.LINENAME
df.head()

,DATE,STATION,LINENAME,DAILY_ENTRIES,DAILY_EXITS,LINE_DENSITY,ZIP_CODE,BOROUGH,CASE_COUNT,HOSPITALIZED_COUNT,...,CASE_COUNT_7DAY_AVG,WEEK_YEAR,HOURS_ELAPSE,zip_code,hospitalizations,borough,time_elapsed,target,station,lines
23727,2022-05-01,AVENUE H,BQ,668.0,1126.0,897.0,11230,Brooklyn,0.0,0.0,...,0.0,2022-05-01,0.0,11230,0.0,Brooklyn,0.0,1794.0,AVENUE H,BQ
23728,2022-05-01,AVENUE J,BQ,1552.0,1333.0,1442.5,11230,Brooklyn,0.0,0.0,...,0.0,2022-05-01,0.0,11230,0.0,Brooklyn,0.0,2885.0,AVENUE J,BQ
23729,2022-05-01,AVENUE M,BQ,1524.0,926.0,1225.0,11230,Brooklyn,0.0,0.0,...,0.0,2022-05-01,0.0,11230,0.0,Brooklyn,0.0,2450.0,AVENUE M,BQ
23730,2022-05-01,AVENUE U,BQ,2035.0,1290.0,1662.5,11229,Brooklyn,0.0,0.0,...,0.0,2022-05-01,0.0,11229,0.0,Brooklyn,0.0,3325.0,AVENUE U,BQ
23731,2022-05-01,AVENUE U,F,0.0,2.0,2.0,11229,Brooklyn,0.0,0.0,...,0.0,2022-05-01,0.0,11229,0.0,Brooklyn,0.0,2.0,AVENUE U,F


In [472]:
MTA_MODEL = MTA_Model()
MTA_MODEL.generate_models(data=df, prominance = 10**5,
                        split=0.2, city=True, boroughs=True, stations=True, lines=True, zip_codes=True)

checkpoint: ./MTA_Model_Checkpoint_City
checkpoint: ./MTA_Model_Checkpoint_Boroughs
checkpoint: ./MTA_Model_Checkpoint_ZipCode
checkpoint: ./MTA_Model_Checkpoint_Lines
checkpoint: ./MTA_Model_Checkpoint_Stations
checkpoint: ./MTA_Model_Checkpoint_Final


In [9]:
MTA_MODEL = load_pickle('./MTA_Model_Checkpoint_Final')

In [13]:
def predict(x):
    time_map = pd.DataFrame(x['time_elapsed'].unique(), columns=['time_elapsed'])
    output = pd.DataFrame(index=time_map.time_elapsed)
    filled_x = x.groupby(['time_elapsed']).agg({'hospitalizations': 'max', 'target': 'sum'}).reset_index()
    # filled_x = pd.merge(time_map, filled_x, on='time_elapsed', how='left')
    filled_x.fillna(0,inplace=True)
    for key in MTA_MODEL.zip_codes.keys():
        output[key] = MTA_MODEL.zip_codes[key].predict(x=filled_x[['time_elapsed', 'hospitalizations']]
                    .groupby('time_elapsed').agg({'hospitalizations': 'max'}).reset_index(), simple=False)
    return output
out1 = predict(future_df)

/tmp/ipykernel_100679/3349951993.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output[key] = MTA_MODEL.zip_codes[key].predict(x=filled_x[['time_elapsed', 'hospitalizations']]
/tmp/ipykernel_100679/3349951993.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  output[key] = MTA_MODEL.zip_codes[key].predict(x=filled_x[['time_elapsed', 'hospitalizations']]
/tmp/ipykernel_100679/3349951993.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

In [31]:
def predict2(x):
    time_map = pd.DataFrame(df['time_elapsed'].unique(), columns=['time_elapsed'])
    output = pd.DataFrame(index=time_map.time_elapsed)
    for key in MTA_MODEL.lines.keys():
        output[key] = MTA_MODEL.lines[key].predict(df[['time_elapsed', 'hospitalizations']])
    return output

out2 = predict2(df)

/tmp/ipykernel_73945/364910292.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[:,'ft_sum'] = array


ValueError: Length of values (6071) does not match length of index (13)

In [74]:
info = {}
for key in MTA_MODEL.lines.keys():
        info[key] ={'Line':key,'Boroughs': ' '.join(MTA_MODEL.lines[key].boroughs),
                    'R-squared': '{:.2e}'.format(MTA_MODEL.lines[key].model_metrics['r_squared'])}
info = pd.DataFrame(info).transpose()
info.Boroughs = info.Boroughs.str.replace('Manhattan', 'Mn')
info.Boroughs = info.Boroughs.str.replace('Queens', 'Qn')
info.Boroughs = info.Boroughs.str.replace('Staten', 'Si')
info.Boroughs = info.Boroughs.str.replace('Bronx', 'Bx')
info.Boroughs = info.Boroughs.str.replace('Brooklyn', 'Bk')


In [15]:
info_zip = {}
for key in MTA_MODEL.zip_codes.keys():
        info_zip[key] ={'Zip Code':key,'Borough': MTA_MODEL.zip_codes[key].boroughs[0],'Stations': len(MTA_MODEL.zip_codes[key].stations),
                        'Lines': len(MTA_MODEL.zip_codes[key].lines), 'r_sq': MTA_MODEL.zip_codes[key].model_metrics['r_squared'],
                    'R-squared': '{:.2e}'.format(MTA_MODEL.zip_codes[key].model_metrics['r_squared'])}
info_stations = pd.DataFrame(info_zip).transpose()
info_stations

,Zip Code,Borough,Stations,Lines,r_sq,R-squared
11230,11230,Brooklyn,6,3,0.684061,6.84e-01
11229,11229,Brooklyn,3,4,0.755759,7.56e-01
11201,11201,Brooklyn,9,14,0.80737,8.07e-01
11235,11235,Brooklyn,3,2,0.667536,6.68e-01
10009,10009,Manhattan,1,1,0.669039,6.69e-01
...,...,...,...,...,...,...
11419,11419,Queens,1,1,0.639771,6.40e-01
11422,11422,Queens,1,1,0.614005,6.14e-01
10463,10463,Bronx,2,1,0.698692,6.99e-01
10471,10471,Bronx,1,1,0.654844,6.55e-01


In [10]:
len(MTA_MODEL.stations)


369

In [597]:
pd.Series(out2.mean()-out.mean()).values

array([798689.10960454, 771283.85410035, 936961.6977167 , 627912.03322949,
       995826.93238362, 903353.62373377, 737446.91521082, 165547.79487246,
       654328.16103673, 321004.9027632 , 853784.04777328, 254857.31514945,
       657472.69458186, 707395.08400252, 697004.23572319, 291137.48154542,
       713798.87707849, 863191.09344488, 716661.37529251, 728932.01184182,
       336630.07711748, 687280.44241553, 794512.89901461])

In [16]:
estimated_growth = pd.DataFrame( out1.mean()/out2.mean()).reset_index()
estimated_growth.columns =['Zip Code', 'Growth']
estimated_growth['Percentage Growth'] = estimated_growth.Growth.round(4)*100
estimated_growth['Absolute Growth'] = (out2.mean()-out1.mean()).values.astype(int)
estimated_growth = pd.merge(estimated_growth, info_stations, on='Zip Code' )
estimated_growth

,Zip Code,Growth,Percentage Growth,Absolute Growth,Borough,Stations,Lines,r_sq,R-squared
0,11230,1.0,100.0,0,Brooklyn,6,3,0.684061,6.84e-01
1,11229,1.0,100.0,0,Brooklyn,3,4,0.755759,7.56e-01
2,11201,1.0,100.0,0,Brooklyn,9,14,0.80737,8.07e-01
3,11235,1.0,100.0,0,Brooklyn,3,2,0.667536,6.68e-01
4,10009,1.0,100.0,0,Manhattan,1,1,0.669039,6.69e-01
...,...,...,...,...,...,...,...,...,...
120,11419,1.0,100.0,0,Queens,1,1,0.639771,6.40e-01
121,11422,1.0,100.0,0,Queens,1,1,0.614005,6.14e-01
122,10463,1.0,100.0,0,Bronx,2,1,0.698692,6.99e-01
123,10471,1.0,100.0,0,Bronx,1,1,0.654844,6.55e-01


In [17]:
pickle_file(estimated_growth, './estimated_growth_zipcode')

In [494]:
metrics = pd.DataFrame(MTA_MODEL.get_metrics()).transpose()
metrics.mse = metrics.mse.astype(float)
metrics.r_squared = metrics.r_squared.astype(float)

In [495]:
# metrics[metrics.type=='Zip Code'].sort_values('r_squared')
# metrics.type.unique()
metrics.describe()

,mse,r_squared
count,5.230000e+02,523.000000
mean,1.043465e+09,0.675487
std,1.393160e+10,0.175091
min,8.163099e+01,-0.554105
25%,7.115368e+05,0.623070
50%,2.825035e+06,0.703661
75%,1.491375e+07,0.787900
max,3.076780e+11,0.932697


In [417]:
df = load_pickle('./data_merged/mta_station_broughs_cases')
# df = df.groupby('DATE','STATION', 'ZIP_CODE', 'LINENAME').sum().reset_index()
df['WEEK_YEAR']= pd.to_datetime(df["DATE"].dt.strftime('%Y-%m'))
# df = df[(df.DATE >= '2020-04-01') & (df.DATE <= '2022-01-01')]
# df = df[df.DATE >= '2022-05-01']
df = df[df.DATE >= '2020-04-01']
# df['HOURS_ELAPSE'] = (df.DATE - pd.to_datetime('2020-04-01')).astype('timedelta64[h]')
df['HOURS_ELAPSE'] = (df.DATE - df.DATE.min()).astype('timedelta64[h]')


df['zip_code'] = df.ZIP_CODE.astype(str)
df['hospitalizations'] = df.HOSPITALIZED_COUNT
df['borough'] = df.BOROUGH
# df['time_elapsed'] = (df.DATE - pd.to_datetime('2020-04-01')).astype('timedelta64[h]')
df['time_elapsed'] = (df.DATE - df.DATE.min()).astype('timedelta64[h]')
df['target'] = df.DAILY_EXITS + df.DAILY_ENTRIES
df['station'] = df.STATION
df['lines'] = df.LINENAME
vis_df = df.groupby('DATE').agg({'time_elapsed':'max', 'hospitalizations': 'max', 'target': 'sum'}).reset_index()
vis_df.head()

,DATE,time_elapsed,hospitalizations,target
0,2020-04-01,0.0,1738.0,1.082347e+06
1,2020-04-02,24.0,1662.0,1.025266e+06
2,2020-04-03,48.0,1703.0,1.025373e+06
3,2020-04-04,72.0,1441.0,6.394701e+05
4,2020-04-05,96.0,1395.0,5.287170e+05


In [390]:
vis_df.describe()

,time_elapsed,hospitalizations,target
count,74.000000,74.000000,7.400000e+01
mean,876.000000,27.540541,4.727097e+06
std,516.139516,18.976690,1.112554e+06
min,0.000000,0.000000,2.436776e+06
25%,438.000000,19.250000,3.538322e+06
50%,876.000000,28.500000,5.265488e+06
75%,1314.000000,42.000000,5.518736e+06
max,1752.000000,76.000000,5.827127e+06


In [353]:
x1, x2, y1, y2 = train_test_split(pd.DataFrame(vis_df['time_elapsed']), vis_df['hospitalizations'], test_size=0.1, random_state=42)
model_cv = LinearRegression()
model_cv.fit(x1, y1) 
pred = model_cv.predict(x2)
print('R-squared: {:.2e}'.format(r2_score(y2, pred)))
print('MSE: {:.2e}'.format(mean_squared_error(y2, pred)))

R-squared: 1.00e+00
MSE: 0.00e+00


In [355]:
estimate_covid = model_cv.predict(pd.DataFrame(vis_df['time_elapsed']))

In [356]:
estimate_covid

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [328]:
vis_df = df.groupby('DATE').sum().reset_index()
vis_df.head()

,DATE,DAILY_ENTRIES,DAILY_EXITS,LINE_DENSITY,ZIP_CODE,CASE_COUNT,HOSPITALIZED_COUNT,DEATH_COUNT,CASE_COUNT_7DAY_AVG,HOURS_ELAPSE,hospitalizations,time_elapsed,target
0,2017-12-30,2.857334e+06,2.312368e+06,2.808571e+06,4944907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.169702e+06
1,2017-12-31,2.435901e+06,2.013643e+06,2.445328e+06,4944907,0.0,0.0,0.0,0.0,11088.0,0.0,11088.0,4.449544e+06
2,2018-01-01,1.898038e+06,1.579894e+06,1.914122e+06,4944907,0.0,0.0,0.0,0.0,22176.0,0.0,22176.0,3.477932e+06
3,2018-01-02,5.046387e+06,3.826915e+06,4.837009e+06,4944907,0.0,0.0,0.0,0.0,33264.0,0.0,33264.0,8.873302e+06
4,2018-01-03,5.531911e+06,4.154126e+06,5.271669e+06,4944907,0.0,0.0,0.0,0.0,44352.0,0.0,44352.0,9.686037e+06


In [9]:
#generate simple model
x1, x2, y1, y2 = train_test_split(pd.DataFrame(vis_df['time_elapsed']), vis_df['target'], test_size=0.1, random_state=42)
model1 = LinearRegression()
model1.fit(x1, y1) 
pred = model1.predict(x2)
print('R-squared: {:.2e}'.format(r2_score(y2, pred)))
print('MSE: {:.2e}'.format(mean_squared_error(y2, pred)))

R-squared: 5.80e-01
MSE: 8.06e+11


In [10]:
# generate more complex model
x1, x2, y1, y2 = train_test_split(vis_df[['time_elapsed', 'hospitalizations']], vis_df['target'], test_size=0.1, random_state=42)
model2 = LinearRegression()
model2.fit(x1, y1) 
pred = model2.predict(x2)
print('R-squared: {:.2e}'.format(r2_score(y2, pred)))
print('MSE: {:.2e}'.format(mean_squared_error(y2, pred)))

R-squared: 5.83e-01
MSE: 8.01e+11


In [426]:
# vis_df = df.groupby('DATE').sum().reset_index()
vis_df['pred_large_model'] = MTA_MODEL.city.predict(vis_df[['time_elapsed', 'hospitalizations']])
vis_df['pred_medium_model'] = model2.predict(vis_df[['time_elapsed', 'hospitalizations']])
vis_df['pred_small_model'] = model1.predict(pd.DataFrame(vis_df['time_elapsed']))
vis_df['estimated_hosp'] = model_cv.predict(pd.DataFrame(vis_df['time_elapsed']))
vis_df.head()

/tmp/ipykernel_39209/743421177.py:48: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,DATE,time_elapsed,hospitalizations,target,pred_large_model,pred_medium_model,pred_small_model,estimated_hosp
0,2020-04-01,0.0,1738.0,1.082347e+06,509969.197462,62299.727465,1.465233e+06,0.0
1,2020-04-02,24.0,1662.0,1.025266e+06,-101905.993273,137621.593382,1.469956e+06,0.0
2,2020-04-03,48.0,1703.0,1.025373e+06,346826.028986,104031.773273,1.474680e+06,0.0
3,2020-04-04,72.0,1441.0,6.394701e+05,117253.070911,352495.293895,1.479403e+06,0.0
4,2020-04-05,96.0,1395.0,5.287170e+05,224873.907727,399891.086472,1.484126e+06,0.0


In [6]:
df = load_pickle('./data_merged/mta_station_broughs_cases')
# df = df.groupby('DATE','STATION', 'ZIP_CODE', 'LINENAME').sum().reset_index()
df['WEEK_YEAR']= pd.to_datetime(df["DATE"].dt.strftime('%Y-%m'))
# df = df[(df.DATE >= '2020-04-01') & (df.DATE <= '2022-01-01')]
# df = df[df.DATE >= '2022-05-01']
df = df[df.DATE >= '2020-04-01']
# df['HOURS_ELAPSE'] = (df.DATE - pd.to_datetime('2020-04-01')).astype('timedelta64[h]')
df['HOURS_ELAPSE'] = (df.DATE - df.DATE.min()).astype('timedelta64[h]')


df['zip_code'] = df.ZIP_CODE.astype(str)
df['hospitalizations'] = df.HOSPITALIZED_COUNT
df['borough'] = df.BOROUGH
# df['time_elapsed'] = (df.DATE - pd.to_datetime('2020-04-01')).astype('timedelta64[h]')
df['time_elapsed'] = (df.DATE - df.DATE.min()).astype('timedelta64[h]')
df['target'] = df.DAILY_EXITS + df.DAILY_ENTRIES
df['station'] = df.STATION
df['lines'] = df.LINENAME
vis_df = df.groupby('DATE').agg({'time_elapsed':'max', 'hospitalizations': 'max', 'target': 'sum'}).reset_index()
vis_df.head()

,DATE,time_elapsed,hospitalizations,target
0,2020-04-01,0.0,1738.0,1.082347e+06
1,2020-04-02,24.0,1662.0,1.025266e+06
2,2020-04-03,48.0,1703.0,1.025373e+06
3,2020-04-04,72.0,1441.0,6.394701e+05
4,2020-04-05,96.0,1395.0,5.287170e+05


In [7]:
days_out = 185

future_df = pd.DataFrame(np.zeros((days_out,4)))
future_df.columns = vis_df.columns
start_date = pd.Timestamp('2022-05-14')
end_date = start_date + pd.DateOffset(days=days_out-1)
future_df['DATE'] = pd.date_range(start= start_date, end=end_date,  periods=None)
future_df.hospitalizations = np.linspace(20, 0, num=days_out)
future_df

,DATE,time_elapsed,hospitalizations,target
0,2022-05-14,0.0,20.000000,0.0
1,2022-05-15,0.0,19.891304,0.0
2,2022-05-16,0.0,19.782609,0.0
3,2022-05-17,0.0,19.673913,0.0
4,2022-05-18,0.0,19.565217,0.0
...,...,...,...,...
180,2022-11-10,0.0,0.434783,0.0
181,2022-11-11,0.0,0.326087,0.0
182,2022-11-12,0.0,0.217391,0.0
183,2022-11-13,0.0,0.108696,0.0


In [9]:
future_df = vis_df[vis_df.DATE >= '2022-11-01']

In [8]:
vis_df = pd.concat([vis_df, future_df])
vis_df['time_elapsed'] = (vis_df.DATE - vis_df.DATE.min()).astype('timedelta64[h]')
vis_df.tail()

,DATE,time_elapsed,hospitalizations,target
180,2022-11-10,22872.0,0.434783,0.0
181,2022-11-11,22896.0,0.326087,0.0
182,2022-11-12,22920.0,0.217391,0.0
183,2022-11-13,22944.0,0.108696,0.0
184,2022-11-14,22968.0,0.000000,0.0


In [10]:
future_df

,DATE,time_elapsed,hospitalizations,target
171,2022-11-01,22656.0,1.413043,0.0
172,2022-11-02,22680.0,1.304348,0.0
173,2022-11-03,22704.0,1.195652,0.0
174,2022-11-04,22728.0,1.086957,0.0
175,2022-11-05,22752.0,0.978261,0.0
176,2022-11-06,22776.0,0.869565,0.0
177,2022-11-07,22800.0,0.760870,0.0
178,2022-11-08,22824.0,0.652174,0.0
179,2022-11-09,22848.0,0.543478,0.0
180,2022-11-10,22872.0,0.434783,0.0


In [465]:
# Create traces
# fig = go.Figure()

def generate_figure(df):
    # df = df[df.TIME_OF_WEEK == 'Weekend']
    fig = make_subplots()
    
    medium = fig.add_trace(go.Scatter(x=df.DATE, y=df.pred_medium_model,
                        mode='lines',
                        # name='Prediction (Date, Hospitalized)',
                        opacity = 0,
                        line=dict(color='navy',width=1),
                        legendgroup="1",  # this can be any string, not just "group"
                        legendgrouptitle_text="Model 2 (R-squared: 5.20e-01)",
                        name="Variables: Date, Hospitalized"),
                        # row = 1,
                        # col = 1
                        )
    small = fig.add_trace(go.Scatter(x=df.DATE, y=df.pred_small_model,
                        mode='lines',
                        # name='Prediction (Date)',
                        opacity = 0,
                        line=dict(color='navy',width=1),
                        legendgroup="2",  # this can be any string, not just "group"
                        legendgrouptitle_text="Model 1 (R-squared: 4.94e-01)",
                        name="Variables: Date"),
                        # row = 2,
                        # col = 1
                        )
    large = fig.add_trace(go.Scatter(x=df.DATE, y=df.pred_large_model,
                        mode='lines',
                        # name='Prediction (Date, Hospitalized, Fourier)',
                        opacity =0 ,
                        line=dict(color='navy',width=1),
                        legendgroup="3",  # this can be any string, not just "group"
                        legendgrouptitle_text="Model 3 (R-squared: 8.12e-01)",
                        name="Date, Hospitalized, FFT*"),
                        # row = 3,
                        # col = 1
                        )
    fig.add_trace(go.Scatter(x=df[df.DATE <= '2022-05-13'].DATE, y=df.target,
                        mode='lines',
                        # name='Actual',
                        opacity = .8,
                        line=dict(color='orange', width=1),
                        legendgroup="4",  # this can be any string, not just "group"
                        legendgrouptitle_text="Actual Subway Traffic",
                        name="Ground-Truth"),
                        # row = 4,
                        # col = 1
                        )


    fig.add_vrect(x0="2022-01-01", x1="2022-05-13", 
              annotation_text="Test Set", annotation_position="bottom right",
              annotation=dict(font_size=20),
              fillcolor="yellow", opacity=0.1, line_width=0)
    fig.add_vrect(x0="2022-05-14", x1="2022-12-31", 
              annotation_text="Forecast", annotation_position="bottom right",
              annotation=dict(font_size=20),
              fillcolor="blue", opacity=0.1, line_width=0)

       # 8458403
    fig.add_hline(y=8458403, line_dash="dash",
       annotation_text="Avergage Entries and Exits 2018-2020", 
       annotation_position="bottom left")
    
    menuadjustment = 0.4
    buttonY = 1.4 - menuadjustment
    menu = []
    menu.append({})
    button = dict(method='restyle',
                  label='Model 1',
                  visible=True,
                  args=[{'visible':[False],
                         'opacity' : 0.1}, [1]],
                  args2 = [{'visible': [True, False, False],
                            'opacity' : 1}, [1, 0, 2]],
                 )
    menu[0]['buttons'] = [button]
    menu[0]['showactive'] = False
    menu[0]['y'] = buttonY

    buttonY = buttonY-menuadjustment
    menu.append({})
    button = dict(method='restyle',
                  label='Model 2',
                  visible=True,
                  args=[{'visible':False,
                         'opacity' : 0.1}, [0]],
                  args2 = [{'visible': [True, False, False],
                            'opacity' : 1}, [0, 1, 2]],
                 )
    menu[1]['buttons'] = [button]
    menu[1]['showactive'] = False
    menu[1]['y'] = buttonY

    buttonY = buttonY-menuadjustment
    menu.append({})
    button = dict(method='restyle',
                  label='Model 3',
                  visible=True,
                  args=[{'visible':False,
                         'opacity' : 0.1}, [2]],
                  args2 = [{'visible': [True, False, False],
                            'opacity' : 1}, [2, 0,1]],
                 )
    menu[2]['buttons'] = [button]
    menu[2]['showactive'] = False
    menu[2]['y'] = buttonY

    # fig.data[1].update(xaxis='x2')
    # fig.update_yaxes(title_text="Predicted")
    # fig.update_yaxes(title_text="Actual")
    # fig.update_layout(legend_title_text='Subway Use')
    fig.update_layout(title='Predicting Subway Use',
                    updatemenus = menu, plot_bgcolor='rgb(233,233,233)')
    for m in fig.layout.updatemenus:
        m['type'] = 'buttons'
    fig['data'][0]['showlegend']=True
    fig.write_html("./images/subway_use_regression_date_hosp_with_fourier.html")
    
    return fig
generate_figure(vis_df).show()

In [106]:
pickle_file(estimated_growth.sort_values('Growth',ascending=False),'./station_expected_growth')

In [73]:
def generate_bar(df):
    fig = px.bar(df, x='Line', y='Percentage Growth',
                hover_data=['Absolute Growth','Boroughs','R-squared'],# color='country',
                # labels={'pop':'population of Canada'}, height=400
                )
    fig.update_yaxes(range=[15,35])
    fig.update_layout(title='Expected Line Growth Over 6 Month (May 2022 to November 2022)')
    fig.write_html("./images/expected_line_growth.html")
    return fig
estimated_growth

fig = generate_bar(estimated_growth.sort_values('Growth',ascending=False))
fig.show()
# estimated_growth.sort_values('Growth',ascending=False)


In [149]:
def generate_bar(df):
    fig = px.bar(df, x='Station', y='Percentage Growth',
                hover_data=['Absolute Growth','Lines','Borough','R-squared'],# color='country',
                # labels={'pop':'population of Canada'}, height=400
                )
    # fig.update_yaxes(range=[15,35])
    fig.update_layout(title='Expected Station Growth Over 6 Month (May 2022 to November 2022)')
    fig.write_html("./images/expected_station_growth.html")
    return fig
estimated_growth

fig = generate_bar(estimated_growth[(
    estimated_growth['Absolute Growth'] >2000) & (estimated_growth.r_sq >.8) & (estimated_growth.Growth >0)    
    ].sort_values('Percentage Growth', ascending=False).iloc[[0,1,2,3,4,-5,-4,-3,-2,-1]])
fig.show()
# estimated_growth.sort_values('Growth',ascending=False)


In [ ]:
import plotly.express as px
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ny_new_york_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

In [159]:
def generate_map(df):
    fig = px.choropleth(df, 
                        geojson=zipcodes, 
                        locations='Zip Code', 
                        color='Growth',
                        color_continuous_scale="Viridis",
                        range_color=(1,5),
                        # featureidkey="properties.ZCTA5CE10",
                        scope="usa",
                        # labels={'Cluster':'Cluster_Category'}
                            )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig
fig = generate_map(estimated_growth)
fig.show()